# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [4]:
problem = '''You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. '''

## 2. Generate the mathematical model

In [5]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [6]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Food and Beverage Production Planning

**Sets and Indices:**

*  `T`: Set of time periods (1 to 12)
*  `F`: Set of food products (1 to 2)
*  `B`: Set of beverage products (1 to 3)

**Parameters:**

*  `FC_f,t`: Fixed cost of producing one unit of food product `f` in time period `t` (from "fixed_cost_production.csv")
*  `VC_f,t`: Variable cost of producing one unit of food product `f` in time period `t` (from "variable_cost_production.csv")
*  `VC_b,t`: Variable cost of producing one liter of beverage product `b` in time period `t` (from "variable_cost_production.csv")
*  `SC_f,t`: Variable cost of storing one unit of food product `f` in time period `t` (from "variable_cost_storage.csv")
*  `SC_b,t`: Variable cost of storing one liter of beverage product `b` in time period `t` (from "variable_cost_storage.csv")
*  `D_f,t`: Demand for food product `f` in time period `t` (from "demand.csv")
*  `D_b,t`: Demand for beverage product `b` in time period `t` (from "demand.csv")
*  `R_f,t`: Revenue from selling one unit of food product `f` in time period `t` (from "revenue.csv")
*  `R_b,t`: Revenue from selling one liter of beverage product `b` in time period `t` (from "revenue.csv")
*  `CapS_f`: Storage capacity for food product `f` (580 for food 1, 687 for food 2)
*  `CapS_b`: Storage capacity for beverage product `b` (599L for beverage 1, 788L for beverage 2, 294L for beverage 3)
*  `CapP_f`: Production capacity for food product `f` (1080 for food 1, 908 for food 2)
*  `CapP_b`: Production capacity for beverage product `b` (408L for beverage 1, 1000L for beverage 2, 403L for beverage 3) 

**Decision Variables:**

*  `P_f,t`: Units of food product `f` produced in time period `t` (integer)
*  `P_b,t`: Liters of beverage product `b` produced in time period `t` (continuous)
*  `S_f,t`: Units of food product `f` stored at the end of time period `t` (integer)
*  `S_b,t`: Liters of beverage product `b` stored at the end of time period `t` (continuous)
*  `Sales_f,t`: Units of food product `f` sold in time period `t` (integer)
*  `Sales_b,t`: Liters of beverage product `b` sold in time period `t` (continuous)

**Objective Function:**

Maximize total profit over all time periods:

```
Maximize: ∑_(t∈T) ∑_(f∈F) (R_f,t * Sales_f,t - FC_f,t * (P_f,t > 0) - VC_f,t * P_f,t - SC_f,t * S_f,t) + ∑_(t∈T) ∑_(b∈B) (R_b,t * Sales_b,t - VC_b,t * P_b,t - SC_b,t * S_b,t) 
```

**Constraints:**

1. **Production Capacity:**
   * For all `f ∈ F`, `t ∈ T`:  `P_f,t ≤ CapP_f`
   * For all `b ∈ B`, `t ∈ T`:  `P_b,t ≤ CapP_b` 

2. **Storage Capacity:**
   * For all `f ∈ F`, `t ∈ T`:  `S_f,t ≤ CapS_f`
   * For all `b ∈ B`, `t ∈ T`:  `S_b,t ≤ CapS_b` 

3. **Inventory Balance:**
   * For all `f ∈ F`, `t ∈ T`:  `S_f,t = S_f,t-1 + P_f,t - Sales_f,t`  (assuming `S_f,0` = 0)
   * For all `b ∈ B`, `t ∈ T`:  `S_b,t = S_b,t-1 + P_b,t - Sales_b,t`  (assuming `S_b,0` = 0)

4. **Demand Satisfaction:**
   * For all `f ∈ F`, `t ∈ T`:  `Sales_f,t ≤ D_f,t` 
   * For all `b ∈ B`, `t ∈ T`:  `Sales_b,t ≤ D_b,t` 

5. **Non-negativity:**
   * All decision variables must be non-negative.

This model incorporates all the provided information and constraints to formulate a comprehensive mathematical optimization problem for the food and beverage production planning scenario. By solving this model, you can determine the optimal production and storage quantities for each product in each time period to maximize the overall profit.


## 3. Generate the pyomo code

In [7]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [8]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100, (1, 2): 110, (1, 3): 105, (1, 4): 108, (1, 5): 112, (1, 6): 106,
    (1, 7): 109, (1, 8): 111, (1, 9): 107, (1, 10): 113, (1, 11): 104, (1, 12): 115,
    (2, 1): 120, (2, 2): 130, (2, 3): 125, (2, 4): 128, (2, 5): 132, (2, 6): 126,
    (2, 7): 129, (2, 8): 131, (2, 9): 127, (2, 10): 133, (2, 11): 124, (2, 12): 135,
}
variable_cost_production = {
    (1, 1): 10, (1, 2): 11, (1, 3): 12, (1, 4): 9, (1, 5): 13, (1, 6): 10,
    (1, 7): 12, (1, 8): 11, (1, 9): 9, (1, 10): 13, (1, 11): 10, (1, 12): 11,
    (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 14, (2, 5): 18, (2, 6): 15,
    (2, 7): 17, (2, 8): 16, (2, 9): 14, (2, 10): 18, (2, 11): 15, (2, 12): 16,
    (1, 1): 2, (1, 2): 2.5, (1, 3): 2.2, (1, 4): 1.8, (1, 5): 2.7, (1, 6): 2.1,
    (1, 7): 2.4, (1, 8): 2.3, (1, 9): 1.9, (1, 10): 2.8, (1, 11): 2, (1, 12): 2.6,
    (2, 1): 3, (2, 2): 3.5, (2, 3): 3.2, (2, 4): 2.8, (2, 5): 3.7, (2, 6): 3.1,
    (2, 7): 3.4, (2, 8): 3.3, (2, 9): 2.9, (2, 10): 3.8, (2, 11): 3, (2, 12): 3.6,
    (3, 1): 1.5, (3, 2): 1.8, (3, 3): 1.6, (3, 4): 1.4, (3, 5): 2, (3, 6): 1.7,
    (3, 7): 1.9, (3, 8): 1.85, (3, 9): 1.55, (3, 10): 2.1, (3, 11): 1.5, (3, 12): 1.95
}
variable_cost_storage = {
    (1, 1): 1, (1, 2): 1.2, (1, 3): 1.1, (1, 4): 0.9, (1, 5): 1.3, (1, 6): 1,
    (1, 7): 1.2, (1, 8): 1.1, (1, 9): 0.9, (1, 10): 1.3, (1, 11): 1, (1, 12): 1.2,
    (2, 1): 1.5, (2, 2): 1.7, (2, 3): 1.6, (2, 4): 1.4, (2, 5): 1.8, (2, 6): 1.5,
    (2, 7): 1.7, (2, 8): 1.6, (2, 9): 1.4, (2, 10): 1.8, (2, 11): 1.5, (2, 12): 1.7,
    (1, 1): 0.5, (1, 2): 0.6, (1, 3): 0.55, (1, 4): 0.45, (1, 5): 0.65, (1, 6): 0.5,
    (1, 7): 0.6, (1, 8): 0.55, (1, 9): 0.45, (1, 10): 0.65, (1, 11): 0.5, (1, 12): 0.6,
    (2, 1): 0.75, (2, 2): 0.85, (2, 3): 0.8, (2, 4): 0.7, (2, 5): 0.9, (2, 6): 0.75,
    (2, 7): 0.85, (2, 8): 0.8, (2, 9): 0.7, (2, 10): 0.9, (2, 11): 0.75, (2, 12): 0.85,
    (3, 1): 0.4, (3, 2): 0.45, (3, 3): 0.42, (3, 4): 0.38, (3, 5): 0.5, (3, 6): 0.44,
    (3, 7): 0.47, (3, 8): 0.46, (3, 9): 0.4, (3, 10): 0.52, (3, 11): 0.4, (3, 12): 0.48
}
demand = {
    (1, 1): 500, (1, 2): 550, (1, 3): 600, (1, 4): 450, (1, 5): 650, (1, 6): 500,
    (1, 7): 580, (1, 8): 620, (1, 9): 480, (1, 10): 680, (1, 11): 520, (1, 12): 590,
    (2, 1): 400, (2, 2): 440, (2, 3): 480, (2, 4): 380, (2, 5): 520, (2, 6): 400,
    (2, 7): 460, (2, 8): 500, (2, 9): 390, (2, 10): 540, (2, 11): 420, (2, 12): 470,
    (1, 1): 250, (1, 2): 275, (1, 3): 300, (1, 4): 225, (1, 5): 325, (1, 6): 250,
    (1, 7): 290, (1, 8): 310, (1, 9): 240, (1, 10): 340, (1, 11): 260, (1, 12): 295,
    (2, 1): 200, (2, 2): 220, (2, 3): 240, (2, 4): 190, (2, 5): 260, (2, 6): 200,
    (2, 7): 230, (2, 8): 250, (2, 9): 195, (2, 10): 270, (2, 11): 210, (2, 12): 235,
    (3, 1): 125, (3, 2): 137.5, (3, 3): 150, (3, 4): 112.5, (3, 5): 162.5, (3, 6): 125,
    (3, 7): 145, (3, 8): 155, (3, 9): 120, (3, 10): 170, (3, 11): 130, (3, 12): 147.5
}
revenue = {
    (1, 1): 200, (1, 2): 220, (1, 3): 240, (1, 4): 180, (1, 5): 260, (1, 6): 200,
    (1, 7): 230, (1, 8): 250, (1, 9): 190, (1, 10): 270, (1, 11): 210, (1, 12): 240,
    (2, 1): 300, (2, 2): 330, (2, 3): 360, (2, 4): 270, (2, 5): 390, (2, 6): 300,
    (2, 7): 345, (2, 8): 375, (2, 9): 285, (2, 10): 405, (2, 11): 315, (2, 12): 360,
    (1, 1): 100, (1, 2): 110, (1, 3): 120, (1, 4): 90, (1, 5): 130, (1, 6): 100,
    (1, 7): 115, (1, 8): 125, (1, 9): 95, (1, 10): 135, (1, 11): 105, (1, 12): 120,
    (2, 1): 150, (2, 2): 165, (2, 3): 180, (2, 4): 135, (2, 5): 195, (2, 6): 150,
    (2, 7): 172.5, (2, 8): 187.5, (2, 9): 142.5, (2, 10): 202.5, (2, 11): 157.5, (2, 12): 180,
    (3, 1): 75, (3, 2): 82.5, (3, 3): 90, (3, 4): 67.5, (3, 5): 97.5, (3, 6): 75,
    (3, 7): 86.25, (3, 8): 93.75, (3, 9): 71.25, (3, 10): 101.25, (3, 11): 78.75, (3, 12): 90
}

CapS_f = {1: 580, 2: 687}
CapS_b = {1: 599, 2: 788, 3: 294}
CapP_f = {1: 1080, 2: 908}
CapP_b = {1: 408, 2: 1000, 3: 403} 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 12)
model.F = pyo.RangeSet(1, 2)
model.B = pyo.RangeSet(1, 3)

# Parameters
model.FC = pyo.Param(model.F, model.T, initialize=fixed_cost_production)
model.VC_f = pyo.Param(model.F, model.T, initialize=variable_cost_production)
model.VC_b = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.SC_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage)
model.SC_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)
model.CapS_f = pyo.Param(model.F, initialize=CapS_f)
model.CapS_b = pyo.Param(model.B, initialize=CapS_b)
model.CapP_f = pyo.Param(model.F, initialize=CapP_f)
model.CapP_b = pyo.Param(model.B, initialize=CapP_b) 

# Decision Variables
model.P_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.P_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.Sales_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.Sales_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    profit = sum(
        model.R_f[f,t] * model.Sales_f[f,t] - model.FC[f,t] * (model.P_f[f,t] >= 1) - model.VC_f[f,t] * model.P_f[f,t] - model.SC_f[f,t] * model.S_f[f,t] 
        for f in model.F for t in model.T
    ) + sum(
        model.R_b[b,t] * model.Sales_b[b,t] - model.VC_b[b,t] * model.P_b[b,t] - model.SC_b[b,t] * model.S_b[b,t]
        for b in model.B for t in model.T
    )
    return profit

model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def production_capacity_f_rule(model, f, t):
    return model.P_f[f,t] <= model.CapP_f[f]
model.production_capacity_f = pyo.Constraint(model.F, model.T, rule=production_capacity_f_rule)

def production_capacity_b_rule(model, b, t):
    return model.P_b[b,t] <= model.CapP_b[b]
model.production_capacity_b = pyo.Constraint(model.B, model.T, rule=production_capacity_b_rule)

def storage_capacity_f_rule(model, f, t):
    return model.S_f[f,t] <= model.CapS_f[f]
model.storage_capacity_f = pyo.Constraint(model.F, model.T, rule=storage_capacity_f_rule)

def storage_capacity_b_rule(model, b, t):
    return model.S_b[b,t] <= model.CapS_b[b]
model.storage_capacity_b = pyo.Constraint(model.B, model.T, rule=storage_capacity_b_rule)

def inventory_balance_f_rule(model, f, t):
    if t == 1:
        return model.S_f[f,t] == model.P_f[f,t] - model.Sales_f[f,t]
    else:
        return model.S_f[f,t] == model.S_f[f,t-1] + model.P_f[f,t] - model.Sales_f[f,t]
model.inventory_balance_f = pyo.Constraint(model.F, model.T, rule=inventory_balance_f_rule)

def inventory_balance_b_rule(model, b, t):
    if t == 1:
        return model.S_b[b,t] == model.P_b[b,t] - model.Sales_b[b,t]
    else:
        return model.S_b[b,t] == model.S_b[b,t-1] + model.P_b[b,t] - model.Sales_b[b,t]
model.inventory_balance_b = pyo.Constraint(model.B, model.T, rule=inventory_balance_b_rule)

def demand_satisfaction_f_rule(model, f, t):
    return model.Sales_f[f,t] <= model.D_f[f,t]
model.demand_satisfaction_f = pyo.Constraint(model.F, model.T, rule=demand_satisfaction_f_rule)

def demand_satisfaction_b_rule(model, b, t):
    return model.Sales_b[b,t] <= model.D_b[b,t]
model.demand_satisfaction_b = pyo.Constraint(model.B, model.T, rule=demand_satisfaction_b_rule)

# Solve
solver = pyo.SolverFactory('cbc')
results = solver.solve(model)

# Print results
print(results)
print("Profit = ", model.profit()) 

# You can access the optimal values of decision variables as follows:
# print("P_f = ", model.P_f.get_values()) 
# print("P_b = ", model.P_b.get_values()) 
# and so on...
```

This code defines the model, sets, parameters, decision variables, objective function, and constraints based on the mathematical formulation. The sample data is used to initialize the parameter values. You should replace this sample data with the actual data from your CSV files. 

The `cbc` solver is used in this code. You can choose a different solver if needed.

After solving, you can access the optimal values of the decision variables to get the optimal production, storage, and sales plan.

## 4. Input problem data and try running the generated code

In [41]:
import pyomo.environ as pyo
# fixed_cost_production = pd.read_csv("fixed_cost_production.csv", usecols=lambda column: column != 0)
# variable_cost_production = pd.read_csv("variable_cost_production.csv", usecols=lambda column: column != 0)
# variable_cost_storage = pd.read_csv("variable_cost_storage.csv", usecols=lambda column: column != 0)
# demand = pd.read_csv("demand.csv", usecols=lambda column: column != 0)
# revenue = pd.read_csv("revenue.csv", usecols=lambda column: column != 0)

# # Transform into dictionary format
# fixed_cost_production = {(i, j): fixed_cost_production.iloc[i-1, j] for i in range(1, len(fixed_cost_production)+1) for j in range(1, len(fixed_cost_production.columns))}
# variable_cost_production = {(i, j): variable_cost_production.iloc[i-1, j] for i in range(1, len(variable_cost_production)+1) for j in range(1, len(variable_cost_production.columns))}
# variable_cost_storage = {(i, j): variable_cost_storage.iloc[i-1, j] for i in range(1, len(variable_cost_storage)+1) for j in range(1, len(variable_cost_storage.columns))}
# demand = {(i, j): demand.iloc[i-1, j] for i in range(1, len(demand)+1) for j in range(1, len(demand.columns))}
# revenue = {(i, j): revenue.iloc[i-1, j] for i in range(1, len(revenue)+1) for j in range(1, len(revenue.columns))}

# # Sample Data (Replace with actual data from CSV files)
fcp={(1, 1): 554.19,
 (1, 2): 572.0,
 (1, 3): 529.18,
 (1, 4): 617.31,
 (1, 5): 533.74,
 (1, 6): 543.39,
 (1, 7): 559.71,
 (1, 8): 571.31,
 (1, 9): 595.68,
 (1, 10): 589.13,
 (1, 11): 540.62,
 (1, 12): 606.22,
 (2, 1): 624.9,
 (2, 2): 562.7,
 (2, 3): 310.2,
 (2, 4): 364.3,
 (2, 5): 537.9,
 (2, 6): 358.44,
 (2, 7): 317.26,
 (2, 8): 368.13,
 (2, 9): 410.52,
 (2, 10): 454.55,
 (2, 11): 463.93,
 (2, 12): 429.31,
 (3, 1): 114.73,
 (3, 2): 134.42,
 (3, 3): 128.27,
 (3, 4): 118.92,
 (3, 5): 119.16,
 (3, 6): 125.07,
 (3, 7): 115.96,
 (3, 8): 135.59,
 (3, 9): 134.14,
 (3, 10): 125.23,
 (3, 11): 142.87,
 (3, 12): 134.33,
 (4, 1): 371.74,
 (4, 2): 360.78,
 (4, 3): 401.41,
 (4, 4): 355.35,
 (4, 5): 301.21,
 (4, 6): 350.6,
 (4, 7): 342.28,
 (4, 8): 284.81,
 (4, 9): 338.89,
 (4, 10): 378.14,
 (4, 11): 271.69,
 (4, 12): 350.92,
 (5, 1): 258.28,
 (5, 2): 433.32,
 (5, 3): 573.87,
 (5, 4): 284.12,
 (5, 5): 390.95,
 (5, 6): 184.32,
 (5, 7): 162.6,
 (5, 8): -28.18,
 (5, 9): 362.46,
 (5, 10): 248.21,
 (5, 11): 252.38,
 (5, 12): 120.68}
fixed_cost_production = {
 (3, 1): 114.73,
 (3, 2): 134.42,
 (3, 3): 128.27,
 (3, 4): 118.92,
 (3, 5): 119.16,
 (3, 6): 125.07,
 (3, 7): 115.96,
 (3, 8): 135.59,
 (3, 9): 134.14,
 (3, 10): 125.23,
 (3, 11): 142.87,
 (3, 12): 134.33,
 (4, 1): 371.74,
 (4, 2): 360.78,
 (4, 3): 401.41,
 (4, 4): 355.35,
 (4, 5): 301.21,
 (4, 6): 350.6,
 (4, 7): 342.28,
 (4, 8): 284.81,
 (4, 9): 338.89,
 (4, 10): 378.14,
 (4, 11): 271.69,
 (4, 12): 350.92,
 (5, 1): 258.28,
 (5, 2): 433.32,
 (5, 3): 573.87,
 (5, 4): 284.12,
 (5, 5): 390.95,
 (5, 6): 184.32,
 (5, 7): 162.6,
 (5, 8): -28.18,
 (5, 9): 362.46,
 (5, 10): 248.21,
 (5, 11): 252.38,
 (5, 12): 120.68
}
variable_cost_production = {
(3, 1): 0.19,
 (3, 2): 0.16,
 (3, 3): 0.27,
 (3, 4): 0.21,
 (3, 5): 0.23,
 (3, 6): 0.22,
 (3, 7): 0.16,
 (3, 8): 0.18,
 (3, 9): 0.28,
 (3, 10): 0.2,
 (3, 11): 0.2,
 (3, 12): 0.2,
 (4, 1): 0.77,
 (4, 2): 0.29,
 (4, 3): 1.49,
 (4, 4): 0.98,
 (4, 5): 0.89,
 (4, 6): 0.82,
 (4, 7): 0.09,
 (4, 8): 0.62,
 (4, 9): 1.31,
 (4, 10): 0.48,
 (4, 11): 1.26,
 (4, 12): 0.62,
 (5, 1): 1.3,
 (5, 2): 1.26,
 (5, 3): 1.25,
 (5, 4): 1.24,
 (5, 5): 1.21,
 (5, 6): 1.3,
 (5, 7): 1.2,
 (5, 8): 1.24,
 (5, 9): 1.21,
 (5, 10): 1.29,
 (5, 11): 1.23,
 (5, 12): 1.27
}
variable_cost_storage = {(3, 1): 10.96,
 (3, 2): 9.63,
 (3, 3): 5.25,
 (3, 4): 17.96,
 (3, 5): 14.44,
 (3, 6): 21.16,
 (3, 7): 5.83,
 (3, 8): 2.45,
 (3, 9): 20.16,
 (3, 10): 13.29,
 (3, 11): 25.79,
 (3, 12): 18.96,
 (4, 1): 7.51,
 (4, 2): 11.85,
 (4, 3): 9.69,
 (4, 4): 11.22,
 (4, 5): 7.55,
 (4, 6): 10.78,
 (4, 7): 9.76,
 (4, 8): 8.93,
 (4, 9): 9.2,
 (4, 10): 11.32,
 (4, 11): 8.51,
 (4, 12): 4.89,
 (5, 1): 23.3,
 (5, 2): 21.57,
 (5, 3): 26.56,
 (5, 4): 28.1,
 (5, 5): 21.75,
 (5, 6): 25.39,
 (5, 7): 25.08,
 (5, 8): 24.53,
 (5, 9): 19.69,
 (5, 10): 26.29,
 (5, 11): 26.37,
 (5, 12): 23.69
}
demand = {(3, 1): 5.68,
 (3, 2): 8.2,
 (3, 3): 7.85,
 (3, 4): 5.83,
 (3, 5): 8.05,
 (3, 6): 6.13,
 (3, 7): 6.38,
 (3, 8): 7.3,
 (3, 9): 6.75,
 (3, 10): 5.83,
 (3, 11): 5.86,
 (3, 12): 5.85,
 (4, 1): 2.71,
 (4, 2): 2.91,
 (4, 3): 2.35,
 (4, 4): 2.98,
 (4, 5): 2.53,
 (4, 6): 2.4,
 (4, 7): 2.91,
 (4, 8): 2.37,
 (4, 9): 1.19,
 (4, 10): 5.14,
 (4, 11): 2.77,
 (4, 12): 2.69,
 (5, 1): 3.93,
 (5, 2): 4.0,
 (5, 3): 4.04,
 (5, 4): 3.99,
 (5, 5): 4.13,
 (5, 6): 3.85,
 (5, 7): 3.94,
 (5, 8): 3.98,
 (5, 9): 3.8,
 (5, 10): 3.97,
 (5, 11): 3.8,
 (5, 12): 3.94
}
revenue = {(3, 1): 56.8,
 (3, 2): 82.0,
 (3, 3): 78.5,
 (3, 4): 58.3,
 (3, 5): 80.5,
 (3, 6): 61.3,
 (3, 7): 63.8,
 (3, 8): 73.0,
 (3, 9): 67.5,
 (3, 10): 58.3,
 (3, 11): 58.6,
 (3, 12): 58.5,
 (4, 1): 27.1,
 (4, 2): 29.1,
 (4, 3): 23.5,
 (4, 4): 29.8,
 (4, 5): 25.3,
 (4, 6): 24.0,
 (4, 7): 29.1,
 (4, 8): 23.7,
 (4, 9): 11.9,
 (4, 10): 51.4,
 (4, 11): 27.7,
 (4, 12): 26.9,
 (5, 1): 39.3,
 (5, 2): 40.0,
 (5, 3): 40.4,
 (5, 4): 39.9,
 (5, 5): 41.3,
 (5, 6): 38.5,
 (5, 7): 39.4,
 (5, 8): 39.8,
 (5, 9): 38.0,
 (5, 10): 39.7,
 (5, 11): 38.0,
 (5, 12): 39.4
}

# # Sample Data (Replace with actual data from CSV files)
fixed_cost_production_f = {(1, 1): 554.19,
 (1, 2): 572.0,
 (1, 3): 529.18,
 (1, 4): 617.31,
 (1, 5): 533.74,
 (1, 6): 543.39,
 (1, 7): 559.71,
 (1, 8): 571.31,
 (1, 9): 595.68,
 (1, 10): 589.13,
 (1, 11): 540.62,
 (1, 12): 606.22,
 (2, 1): 624.9,
 (2, 2): 562.7,
 (2, 3): 310.2,
 (2, 4): 364.3,
 (2, 5): 537.9,
 (2, 6): 358.44,
 (2, 7): 317.26,
 (2, 8): 368.13,
 (2, 9): 410.52,
 (2, 10): 454.55,
 (2, 11): 463.93,
 (2, 12): 429.31
}
variable_cost_production_f = {(1, 1): 2.86,
 (1, 2): 2.93,
 (1, 3): 3.44,
 (1, 4): 3.24,
 (1, 5): 3.18,
 (1, 6): 3.16,
 (1, 7): 3.05,
 (1, 8): 3.74,
 (1, 9): 2.99,
 (1, 10): 2.72,
 (1, 11): 3.14,
 (1, 12): 3.0,
 (2, 1): 9.09,
 (2, 2): 5.15,
 (2, 3): 2.85,
 (2, 4): 4.37,
 (2, 5): 3.19,
 (2, 6): 2.01,
 (2, 7): 6.21,
 (2, 8): 4.18,
 (2, 9): 3.26,
 (2, 10): 2.81,
 (2, 11): 1.8,
 (2, 12): 5.01
}
variable_cost_storage_f = {
(1, 1): 49.25,
 (1, 2): 43.77,
 (1, 3): 43.83,
 (1, 4): 46.85,
 (1, 5): 43.49,
 (1, 6): 45.11,
 (1, 7): 46.18,
 (1, 8): 45.08,
 (1, 9): 49.74,
 (1, 10): 44.29,
 (1, 11): 45.22,
 (1, 12): 48.2,
 (2, 1): 87.99,
 (2, 2): 66.04,
 (2, 3): 74.1,
 (2, 4): 65.13,
 (2, 5): 78.07,
 (2, 6): 76.73,
 (2, 7): 84.31,
 (2, 8): 92.07,
 (2, 9): 88.65,
 (2, 10): 60.96,
 (2, 11): 82.86,
 (2, 12): 77.58
}
demand_f = {(1, 1): 4.61,
 (1, 2): 4.93,
 (1, 3): 4.92,
 (1, 4): 4.71,
 (1, 5): 4.87,
 (1, 6): 4.6,
 (1, 7): 5.02,
 (1, 8): 4.86,
 (1, 9): 4.87,
 (1, 10): 4.9,
 (1, 11): 4.67,
 (1, 12): 5.2,
 (2, 1): 3.79,
 (2, 2): 4.2,
 (2, 3): 2.81,
 (2, 4): 3.89,
 (2, 5): 2.52,
 (2, 6): 5.21,
 (2, 7): 4.5,
 (2, 8): 4.1,
 (2, 9): 1.84,
 (2, 10): 4.66,
 (2, 11): 4.53,
 (2, 12): 4.16
}
revenue_f = {(1, 1): 46.1,
 (1, 2): 49.3,
 (1, 3): 49.2,
 (1, 4): 47.1,
 (1, 5): 48.7,
 (1, 6): 46.0,
 (1, 7): 50.2,
 (1, 8): 48.6,
 (1, 9): 48.7,
 (1, 10): 49.0,
 (1, 11): 46.7,
 (1, 12): 52.0,
 (2, 1): 37.9,
 (2, 2): 42.0,
 (2, 3): 28.1,
 (2, 4): 38.9,
 (2, 5): 25.2,
 (2, 6): 52.1,
 (2, 7): 45.0,
 (2, 8): 41.0,
 (2, 9): 18.4,
 (2, 10): 46.6,
 (2, 11): 45.3,
 (2, 12): 41.6,
}

CapS_f = {1: 580, 2: 687}
CapS_b = {3: 599, 4: 788, 5: 294}
CapP_f = {1: 1080, 2: 908}
CapP_b = {3: 408, 4: 1000, 5: 403} 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 13)
model.F = [1,2]
model.B = [3,4,5]
model.C = [1,2,3,4,5]

# Parameters
model.FC = pyo.Param(model.C, model.T, initialize=fcp)
model.VC_f = pyo.Param(model.F, model.T, initialize=variable_cost_production_f)
model.VC_b = pyo.Param(model.B, model.T, initialize=variable_cost_production)
model.SC_f = pyo.Param(model.F, model.T, initialize=variable_cost_storage_f)
model.SC_b = pyo.Param(model.B, model.T, initialize=variable_cost_storage)
model.D_f = pyo.Param(model.F, model.T, initialize=demand_f)
model.D_b = pyo.Param(model.B, model.T, initialize=demand)
model.R_f = pyo.Param(model.F, model.T, initialize=revenue_f)
model.R_b = pyo.Param(model.B, model.T, initialize=revenue)
model.CapS_f = pyo.Param(model.F, initialize=CapS_f)
model.CapS_b = pyo.Param(model.B, initialize=CapS_b)
model.CapP_f = pyo.Param(model.F, initialize=CapP_f)
model.CapP_b = pyo.Param(model.B, initialize=CapP_b) 

# Decision Variables
model.P_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.P_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.S_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.S_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)
model.Sales_f = pyo.Var(model.F, model.T, domain=pyo.NonNegativeIntegers)
model.Sales_b = pyo.Var(model.B, model.T, domain=pyo.NonNegativeReals)

# Objective Function
def objective_rule(model):
    profit = sum(
        model.R_f[f,t] * model.Sales_f[f,t] - model.FC[f,t] * (model.P_f[f,t] >= 1) - model.VC_f[f,t] * model.P_f[f,t] - model.SC_f[f,t] * model.S_f[f,t] 
        for f in model.F for t in model.T) + sum(model.R_b[b,t] * model.Sales_b[b,t] - model.VC_b[b,t] * model.P_b[b,t] - model.SC_b[b,t] * model.S_b[b,t]
        for b in model.B for t in model.T)
    return profit

model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints
def production_capacity_f_rule(model, f, t):
    return model.P_f[f,t] <= model.CapP_f[f]
model.production_capacity_f = pyo.Constraint(model.F, model.T, rule=production_capacity_f_rule)

def production_capacity_b_rule(model, b, t):
    return model.P_b[b,t] <= model.CapP_b[b]
model.production_capacity_b = pyo.Constraint(model.B, model.T, rule=production_capacity_b_rule)

def storage_capacity_f_rule(model, f, t):
    return model.S_f[f,t] <= model.CapS_f[f]
model.storage_capacity_f = pyo.Constraint(model.F, model.T, rule=storage_capacity_f_rule)

def storage_capacity_b_rule(model, b, t):
    return model.S_b[b,t] <= model.CapS_b[b]
model.storage_capacity_b = pyo.Constraint(model.B, model.T, rule=storage_capacity_b_rule)

def inventory_balance_f_rule(model, f, t):
    if t == 1:
        return model.S_f[f,t] == model.P_f[f,t] - model.Sales_f[f,t]
    else:
        return model.S_f[f,t] == model.S_f[f,t-1] + model.P_f[f,t] - model.Sales_f[f,t]
model.inventory_balance_f = pyo.Constraint(model.F, model.T, rule=inventory_balance_f_rule)

def inventory_balance_b_rule(model, b, t):
    if t == 1:
        return model.S_b[b,t] == model.P_b[b,t] - model.Sales_b[b,t]
    else:
        return model.S_b[b,t] == model.S_b[b,t-1] + model.P_b[b,t] - model.Sales_b[b,t]
model.inventory_balance_b = pyo.Constraint(model.B, model.T, rule=inventory_balance_b_rule)

def demand_satisfaction_f_rule(model, f, t):
    return model.Sales_f[f,t] <= model.D_f[f,t]
model.demand_satisfaction_f = pyo.Constraint(model.F, model.T, rule=demand_satisfaction_f_rule)

def demand_satisfaction_b_rule(model, b, t):
    return model.Sales_b[b,t] <= model.D_b[b,t]
model.demand_satisfaction_b = pyo.Constraint(model.B, model.T, rule=demand_satisfaction_b_rule)

# Solve
solver = pyo.SolverFactory('cbc')
results = solver.solve(model)

# Print results
print(results)
print("Profit = ", model.profit()) 

ERROR: Rule failed when generating expression for Objective profit with index
None: TypeError: unsupported operand type(s) for *: 'float' and
'InequalityExpression'
ERROR: Constructing component 'profit' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'float' and
        'InequalityExpression'


TypeError: unsupported operand type(s) for *: 'float' and 'InequalityExpression'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [42]:
print(response.text)

## Mathematical Optimization Model for Food and Beverage Production Planning

**Sets and Indices:**

*  `T`: Set of time periods (1 to 12)
*  `F`: Set of food products (1 to 2)
*  `B`: Set of beverage products (1 to 3)

**Parameters:**

*  `FC_f,t`: Fixed cost of producing one unit of food product `f` in time period `t` (from "fixed_cost_production.csv")
*  `VC_f,t`: Variable cost of producing one unit of food product `f` in time period `t` (from "variable_cost_production.csv")
*  `VC_b,t`: Variable cost of producing one liter of beverage product `b` in time period `t` (from "variable_cost_production.csv")
*  `SC_f,t`: Variable cost of storing one unit of food product `f` in time period `t` (from "variable_cost_storage.csv")
*  `SC_b,t`: Variable cost of storing one liter of beverage product `b` in time period `t` (from "variable_cost_storage.csv")
*  `D_f,t`: Demand for food product `f` in time period `t` (from "demand.csv")
*  `D_b,t`: Demand for beverage product `b` in time period `

In [43]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from CSV files)
fixed_cost_production = {
    (1, 1): 100, (1, 2): 110, (1, 3): 105, (1, 4): 108, (1, 5): 112, (1, 6): 106,
    (1, 7): 109, (1, 8): 111, (1, 9): 107, (1, 10): 113, (1, 11): 104, (1, 12): 115,
    (2, 1): 120, (2, 2): 130, (2, 3): 125, (2, 4): 128, (2, 5): 132, (2, 6): 126,
    (2, 7): 129, (2, 8): 131, (2, 9): 127, (2, 10): 133, (2, 11): 124, (2, 12): 135,
}
variable_cost_production = {
    (1, 1): 10, (1, 2): 11, (1, 3): 12, (1, 4): 9, (1, 5): 13, (1, 6): 10,
    (1, 7): 12, (1, 8): 11, (1, 9): 9, (1, 10): 13, (1, 11): 10, (1, 12): 11,
    (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 14, (2, 5): 18, (2, 6): 15,
    (2, 7): 17, (2, 8): 16, (2, 9): 14, (2, 10): 18, (2, 11): 15, (2, 12): 16,
    (1, 1): 2, (1, 2): 2.5, (1, 3): 2.2, (1, 4): 1.8, (1, 5): 2.7, (1, 6): 2.1,
    (1, 7): 2.4, (1, 8): 2.3, (1, 9): 1.9, (1, 10): 2.8, (1, 11): 2, (1, 12): 2.6,
    (2, 1): 3, (2, 2): 3.5, (2, 3): 3.2, (2